In [1]:
import os
import numpy as np
import cv2
from math import log10
import matplotlib.pyplot as plt

def load_image(image_path, is_bayer=False):
    """
    Load image with proper handling for Bayer and RGB images
    
    Parameters:
        image_path (str): Path to the image
        is_bayer (bool): If True, load as Bayer image
        
    Returns:
        numpy.ndarray: Loaded image
    """
    if is_bayer:
        # Load Bayer image (8-bit)
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise ValueError(f"Could not load Bayer image: {image_path}")
    else:
        # Load RGB image
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not load RGB image: {image_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    return img

def calculate_psnr(original, processed):
    """
    Calculate PSNR between two images normalized to [0, 1]
    """
    mse = np.mean((original - processed) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * log10(1.0 / np.sqrt(mse))

def calculate_channel_psnr(original, processed):
    """
    Calculate per-channel PSNR and CPSNR for RGB images normalized to [0, 1]
    """
    psnr_r = calculate_psnr(original[:,:,0], processed[:,:,0])
    psnr_g = calculate_psnr(original[:,:,1], processed[:,:,1])
    psnr_b = calculate_psnr(original[:,:,2], processed[:,:,2])
    cpsnr = (psnr_r + psnr_g + psnr_b) / 3
    return psnr_r, psnr_g, psnr_b, cpsnr

def process_image(bayer_path, gt_path):
    """
    Process a single image pair (Bayer and ground truth)
    """
    # Load images
    bayer_img = load_image(bayer_path, is_bayer=True)
    gt_img = load_image(gt_path, is_bayer=False)
    
    # Normalize images to [0, 1] for PSNR calculation
    bayer_float = bayer_img.astype(np.float32) / 255.0
    gt_float = gt_img.astype(np.float32) / 255.0
    
    # Apply bilinear demosaicing
    bilinear = cv2.demosaicing(bayer_img, cv2.COLOR_BAYER_BG2RGB)
    bilinear_float = bilinear.astype(np.float32) / 255.0
    
    # Apply adaptive bilinear demosaicing
    adaptive = cv2.demosaicing(bayer_img, cv2.COLOR_BAYER_BG2RGB_EA)
    adaptive_float = adaptive.astype(np.float32) / 255.0
    
    # Calculate metrics
    bilinear_psnr = calculate_psnr(gt_float, bilinear_float)
    adaptive_psnr = calculate_psnr(gt_float, adaptive_float)
    
    bilinear_channel_psnr = calculate_channel_psnr(gt_float, bilinear_float)
    adaptive_channel_psnr = calculate_channel_psnr(gt_float, adaptive_float)
    
    # Save results
    output_dir = 'result_kodak_bilinear'
    os.makedirs(output_dir, exist_ok=True)
    
    base_name = os.path.basename(bayer_path)
    cv2.imwrite(os.path.join(output_dir, f'bilinear_{base_name}'),
                cv2.cvtColor(bilinear, cv2.COLOR_RGB2BGR))
    
    return {
        'bilinear': {
            'overall_psnr': bilinear_psnr,
            'channel_psnr': bilinear_channel_psnr
        },
        'adaptive': {
            'overall_psnr': adaptive_psnr,
            'channel_psnr': adaptive_channel_psnr
        }
    }

def analyze_dataset(dataset_path='../dataset/kodak'):
    """
    Process all images in the dataset
    """
    input_dir = os.path.join(dataset_path, 'input')
    gt_dir = os.path.join(dataset_path, 'groundtruth')
    
    if not os.path.exists(input_dir) or not os.path.exists(gt_dir):
        raise ValueError(f"Input or ground truth directory not found in {dataset_path}")
    
    # Get all input files
    image_files = [f for f in os.listdir(input_dir) if f.endswith('.png')]
    image_files.sort()
    
    if not image_files:
        raise ValueError(f"No PNG files found in {input_dir}")
    
    results = []
    for img_file in image_files:
        print(f"Processing {img_file}...")
        
        input_path = os.path.join(input_dir, img_file)
        gt_path = os.path.join(gt_dir, img_file)
        
        if not os.path.exists(gt_path):
            print(f"Warning: Ground truth not found for {img_file}, skipping...")
            continue
        
        try:
            result = process_image(input_path, gt_path)
            results.append({
                'file': img_file,
                'metrics': result
            })
        except Exception as e:
            print(f"Error processing {img_file}: {str(e)}")
            continue
    
    if not results:
        raise ValueError("No images were successfully processed")
    
    # Calculate average metrics
    avg_bilinear_psnr = np.mean([r['metrics']['bilinear']['overall_psnr'] for r in results])
    avg_adaptive_psnr = np.mean([r['metrics']['adaptive']['overall_psnr'] for r in results])
    
    avg_bilinear_channel = np.mean([r['metrics']['bilinear']['channel_psnr'] for r in results], axis=0)
    avg_adaptive_channel = np.mean([r['metrics']['adaptive']['channel_psnr'] for r in results], axis=0)
    
    print("\nAverage Results:")
    print("-" * 50)
    print(f"Standard Bilinear PSNR: {avg_bilinear_psnr:.2f} dB")
    print(f"Channel PSNR - CPSNR: {avg_bilinear_channel[3]:.2f}, R: {avg_bilinear_channel[0]:.2f}, G: {avg_bilinear_channel[1]:.2f}, B: {avg_bilinear_channel[2]:.2f}")
    print(f"\nAdaptive Bilinear PSNR: {avg_adaptive_psnr:.2f} dB")
    print(f"Channel PSNR - CPSNR: {avg_adaptive_channel[3]:.2f}, R: {avg_adaptive_channel[0]:.2f}, G: {avg_adaptive_channel[1]:.2f}, B: {avg_adaptive_channel[2]:.2f}")
    
    return None

In [2]:
analyze_dataset()

Processing kodim01.png...
Processing kodim02.png...
Processing kodim03.png...
Processing kodim04.png...
Processing kodim05.png...
Processing kodim06.png...
Processing kodim07.png...
Processing kodim08.png...
Processing kodim09.png...
Processing kodim10.png...
Processing kodim11.png...
Processing kodim12.png...
Processing kodim13.png...
Processing kodim14.png...
Processing kodim15.png...
Processing kodim16.png...
Processing kodim17.png...
Processing kodim18.png...
Processing kodim19.png...
Processing kodim20.png...
Processing kodim21.png...
Processing kodim22.png...
Processing kodim23.png...
Processing kodim24.png...

Average Results:
--------------------------------------------------
Standard Bilinear PSNR: 29.12 dB
Channel PSNR - CPSNR: 29.34, R: 28.15, G: 31.25, B: 28.62

Adaptive Bilinear PSNR: 29.08 dB
Channel PSNR - CPSNR: 29.29, R: 28.15, G: 31.10, B: 28.62
